<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 7 - Introducción a LightFM</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Preprocesamiento](#section2)
* [3. Modelo](#section3)
* [4. Validación cruzada](#section4)
* [5. Entrenamiento](#section5)
* [6. Evaluación](#section6)
* [7. Predicciones](#section7)
* [8. Enlaces de interés](#section8)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
#!pip install lightfm
#!pip install sklearn

In [3]:
import html
import numpy as np
import os
import pandas as pd
import time

In [4]:
NUM_THREADS = 8

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

En esta libreta vamos a presentar el modelo ***LightFM***, el cual usaremos para construir nuestro sistema de continuación de playlists. Junto a la explicación de las funciones que ofrece, vamos a emplear el conjunto de datos *GoodBooks* que incluimos como ejemplo para ver con más detalle cómo funciona *LightFM*.

<br>

---

<br>

<a id="section11"></a>
### <font color="#92002A">1.1 - ¿Qué es *LightFM*?</font>
<br>

*LightFM* es una implementación en *Python* de una serie de algoritmos de recomendación para la retroalimentación implícita y explícita, incluida la implementación eficiente de las funciones de pérdida BPR y WARP. Es fácil de usar, rápido (a través de la estimación de modelos multiproceso) y produce resultados de alta calidad.

También permite incorporar metadatos de elementos y usuarios en los algoritmos tradicionales de factorización matricial, representando a cada usuario y elemento como la suma de las representaciones latentes de sus características, permitiendo así que las recomendaciones se generalicen a nuevos elementos (a través de las características del elemento) y a nuevos usuarios (a través de las características del usuario).

<br>

<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: *LightFM* hace uso de la librería **OpenMP**. En el caso de *Windows* y *macOS* no podremos emplear más de un hilo para realizar diferentes tareas (como la de entrenamiento), por lo que en caso de requerir el uso de múltiples hilos es necesario emplear un sistema operativo *Linux*.
</div>

<br>

---


<br>


<a id="section12"></a>
### <font color="#92002A">1.2 - Motivo de la elección</font>

<br>

La razón por la cual hemos optado por la librería *LightFM* es el uso de un sistema de recomendación hibrido, el cual resulta de combinar el filtrado colaborativo y el filtrado basado en contenido.

Otra de las razones por las cuales hemos optado por ella, es el problema del *arranque en frio*. En nuestro proyecto, debemos ser capaces de realizar una recomendación basada en las características del usuario y sin tener ninguna valoración previa de éste.


<br>

----

<a id="section13"></a>
### <font color="#92002A">1.3 - DataSet GoodBooks</font>
<br>

*LightFM* proporciona dos *DataSets* con los que podemos realizar pruebas (uno de películas y otro bursátil). Puesto que ninguno de ellos contiene información sobre los usuarios (*features*) y también nos interesa estudiar la parte del preprocesado de los datos, vamos a hacer uso del conjunto de datos *GoodBooks*, que podemos encontrar en el siguiente [enlace](http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip).

Dicho conjunto de datos nos proporciona tres ficheros:

* `BX-Users.csv`, el cual nos proporciona información sobre los usuarios.
* `BX-Books.csv` contiene información acerca de los libros empleados en el conjunto.
* `BX-Books-Ratings.csv` proporciona las calificaciones que los usuarios han realizado sobre el conjunto de libros.

<br>

Para facilitar el proceso de obtención, hemos creado un módulo de Python llamado ***goodbooks***. Tras importarlo a nuestro proyecto, podemos obtener el conjunto de usuarios, libros y calificaciones de la siguiente manera:
* **get_data()**: devuelve tres objetos DictReader en el siguiente orden:
        1. Características de usuarios
        2. Características de libros
        3. Calificaciones
* **get_user_features()**: devuelve las características de los usuarios.
* **get_book_features()**: devuelve las características de los libros.
* **get_ratings()**: devuelve las calificaciones realizadas.

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: *DictReader* crea un objeto el cuál mapea la información leída a un diccionario.
</div>

In [5]:
import modules.goodbooks as goodbooks

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Preprocesamiento</font>

<br>

Para comenzar a usar *LightFM*, debemos preparar los datos que tenemos según el formato que nos indica la librería. Para esta tarea haremos uso de la clase `lightfm.data.Dataset`, la cual nos ayudará con la construcción.

Los datos que debemos obtener, son los siguientes:
* **Interacciones**.
* **Características del usuario**.
* **Características del item**.


<br>

Antes de crear las matrices que hemos mencionado, debemos crear un objeto de tipo `Dataset`: 

In [6]:
from lightfm.data import Dataset

books_dataset = Dataset()

c:\pythonenv\finaltfg\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


Tras crear el objeto `books_dataset`, mapeamos los identificadores de los usuarios y los items de nuestro conjunto mediante la función **fit**:

In [7]:
books_dataset.fit((x['User-ID'] for x in goodbooks.get_ratings()), 
                  (x['ISBN'] for x in goodbooks.get_ratings()))

In [8]:
num_users, num_items = books_dataset.interactions_shape()
print('num users: {}, num_items {}.'.format(num_users, num_items))

num users: 105283, num_items 340553.


A continuación, incorporamos los items (libros) y los usuarios junto a sus correspondientes características:

In [9]:
books_dataset.fit_partial(items=(x['ISBN'] for x in goodbooks.get_book_features()),
                          item_features=(x['Book-Author'] for x in goodbooks.get_book_features()),
                          users=(x['User-ID'] for x in goodbooks.get_user_features()),
                          user_features=(x['Age'] for x in goodbooks.get_user_features()))

Una vez realizado el proceso anterior, ya podemos crear las matrices que necesitamos para poder construir un modelo de *LightFM*.

Las interacciones consisten en dos matrices de tipo `sparse.coo_matrix`, cuyo tamaño es *(num_users , num_items)*. En la primera matriz, se indica la interacción de los usuarios con los items (cuyos valores posibles son *1*, si existe interacción, y *0* en caso contrario). La segunda matriz es similar, salvo que en vez de almacenar si existe interacción o no, almacenamos el *peso* de ésta.

Mediante la función [***build_interactions***()](http://lyst.github.io/lightfm/docs/lightfm.data.html#lightfm.data.Dataset.build_interactions), que esta disponible dentro de la clase `Dataset`, podemos construir dichas matrices.

In [10]:
(interactions, weights) = books_dataset.build_interactions(((x['User-ID'], x['ISBN'],int(x['Book-Rating']))
                                                            for x in goodbooks.get_ratings()))

print(repr(interactions))

<278858x341762 sparse matrix of type '<class 'numpy.int32'>'
	with 1149780 stored elements in COOrdinate format>


Hasta este punto ya tenemos lo necesario para construir un modelo *LightFM*, puesto que ya hemos codificado las interacciones entre usuarios y los items.

Como tenemos características de los usuarios y de los items, también podemos crear las correspondientes matrices de características empleando las funciones [***build_item_features***()](http://lyst.github.io/lightfm/docs/lightfm.data.html#lightfm.data.Dataset.build_item_features) y [***build_user_features***()](http://lyst.github.io/lightfm/docs/lightfm.data.html#lightfm.data.Dataset.build_user_features):

In [11]:
item_features = books_dataset.build_item_features(((x['ISBN'], [x['Book-Author']])
                                                   for x in goodbooks.get_book_features()))
print(repr(item_features))

<341762x443805 sparse matrix of type '<class 'numpy.float32'>'
	with 613141 stored elements in Compressed Sparse Row format>


In [12]:
user_features = books_dataset.build_user_features(((x['User-ID'], [x['Age']])
                                                   for x in goodbooks.get_user_features()))
print(repr(user_features))

<278858x278860 sparse matrix of type '<class 'numpy.float32'>'
	with 557716 stored elements in Compressed Sparse Row format>


In [13]:
item_labels = np.array([x['ISBN'] for x in goodbooks.get_ratings()])
item_labels[0:10]

array(['034545104X', '0155061224', '0446520802', '052165615X',
       '0521795028', '2080674722', '3257224281', '0600570967',
       '038550120X', '342310538'], dtype='<U14')

In [14]:
user_labels = np.array([x['User-ID'] for x in goodbooks.get_ratings()])
user_labels[0:10]

array(['276725', '276726', '276727', '276729', '276729', '276733',
       '276736', '276737', '276744', '276745'], dtype='<U6')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Modelo</font>

<br>

Para crear un modelo *LightFM*, emplearemos la siguiente clase `lightfm.LightFM`. La definición del modelo se realiza de la siguiente manera:


```python
LightFM(no_components=10, k=5, n=10, learning_schedule=’adagrad’, loss=’logistic’, 
        learning_rate=0.05, rho=0.95, epsilon=1e-06, item_alpha=0.0, user_alpha=0.0, 
        max_sampled=10, random_state=None)[source]
```
<br>

A continuación, pasamos a describir algunos de sus parámetros más importantes (el resto podemos encontrarlo en la [documentación oficial](http://lyst.github.io/lightfm/docs/lightfm.html#lightfm.LightFM):

* *Función de pérdida (loss)*. Tenemos disponibles las siguientes funciones:
    * **logistic**: Útil cuando en las interacciones encontramos valores positivos (1) y negativos (-1).
    * **BPR**: *Bayesian Personalised Ranking*. Maximiza la diferencia de predicción entre un ejemplo positivo y un ejemplo negativo elegido al azar. Es útil cuando sólo hay interacciones positivas y se desea optimizar ROC AUC.
    * **WARP**: *Weighted Approximate-Rank Pairwise*. Maximiza el rango de ejemplos positivos al muestrear repetidamente ejemplos negativos hasta que se encuentre el rango lo incumpla. Es útil cuando solo hay interacciones positivas y se desea optimizar la parte superior de la lista de recomendaciones (precision@k).
    * **k-OS WARP**: *k-th order statistic loss*. Una modificación de WARP que utiliza el k-ésimo ejemplo positivo para cualquier usuario dado como base para actualizaciones por pares.
    
<br>

* *Learning Schedule (learning_schedule*. Nos encontramos con dos opciones disponibles:
    * [**adagrag**](https://ruder.io/optimizing-gradient-descent/index.html#adagrad).
    * [**adadelta**](https://ruder.io/optimizing-gradient-descent/index.html#adadelta).

<br>

Para el ejemplo en el cual nos encontramos, vamos a emplear la función de perdida *WARP* y vamos a dejar el resto de los parámetros a su valor por defecto:

In [15]:
from lightfm import LightFM

model = LightFM(loss='bpr')

print(model.get_params())

{'loss': 'bpr', 'learning_schedule': 'adagrad', 'no_components': 10, 'learning_rate': 0.05, 'k': 5, 'n': 10, 'rho': 0.95, 'epsilon': 1e-06, 'max_sampled': 10, 'item_alpha': 0.0, 'user_alpha': 0.0, 'random_state': RandomState(MT19937) at 0x213E8FEF8C8}


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section4"></a>
## <font color="#92002A">4 - Validación cruzada</font>

<br>

*LigtFM* Incorpora una clase que nos permite realizar el proceso de validación cruzada. En el módulo `lightfm.cross_validation` nos encontramos con la función ***random_train_test_split***(), la cual toma un conjunto de interacción y lo divide en dos conjuntos disjuntos, un conjunto de entrenamiento y un conjunto de prueba. Hay que tener en cuenta que no se hace ningún esfuerzo para asegurarse de que todos los elementos y usuarios con interacciones en el conjunto de pruebas también tengan interacciones en el conjunto de entrenamiento (lo que puede conducir a un problema parcial de *arranque en frío* en el conjunto de prueba.
<br>

In [16]:
from lightfm.cross_validation import random_train_test_split

train, test = random_train_test_split(interactions, test_percentage=0.2, random_state=None)

In [17]:
print(repr(train))

<278858x341762 sparse matrix of type '<class 'numpy.int32'>'
	with 919824 stored elements in COOrdinate format>


In [18]:
print(repr(test))

<278858x341762 sparse matrix of type '<class 'numpy.int32'>'
	with 229956 stored elements in COOrdinate format>


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section5"></a>
## <font color="#92002A">5 - Entrenamiento</font>

<br>

El proceso de entrenamiento podemos realizarlo de forma completa [(*fit*)](http://lyst.github.io/lightfm/docs/lightfm.html#lightfm.LightFM.fit) y de forma parcial [(*fit_partial*)](http://lyst.github.io/lightfm/docs/lightfm.html#lightfm.LightFM.fit_partial). Si optamos por esta última, podemos optar por realizar el entrenamiento de forma parcial (en varias etapas).

<br>

```python
fit(interactions, user_features=None, item_features=None, sample_weight=None, epochs=1, num_threads=1, verbose=False
```

<br>

Para el dataset que estamos empleando como ejemplo, vamos a realizar el proceso de entrenamiento de forma completa:

In [19]:
model.fit(train, item_features=item_features, user_features=user_features, 
          verbose=True, epochs=10, num_threads=NUM_THREADS)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section6"></a>
## <font color="#92002A">6 - Evaluación</font>

<br>

Dentro del módulo `lightfm.evaluation`, podemos encontrar funciones de evaluación adecuadas para juzgar el rendimiento de un modelo *LightFM* entrenado:

* **precision_at_k**: Mide la precisión empleando la métrica *k* para un modelo: la fracción de positivos conocidos en las primeras *k* posiciones de la lista clasificada de resultados.
* **recall_at_k**: Mide el número de elementos positivos en las primeras k posiciones de la lista clasificada de resultados dividido por el número de elementos positivos en el período de prueba.
* **auc_score**: Mide la probabilidad de que un ejemplo positivo elegido al azar tenga una puntuación más alta que un ejemplo negativo elegido al azar.
* **reciprocal_rank**: 1 / (rango del ejemplo positivo mejor clasificado).

En las métricas anteriores, la puntuación más alta es *1* y la más baja es *0*.


<br>

Para nuestro modelo, vamos a probar las métricas *precision_at_k*, *recall_at_k* y *auc_score*:

In [ ]:
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

precision = precision_at_k(model, num_threads=NUM_THREADS, k=10,
                           test_interactions=test,train_interactions=train,                                 
                           item_features=item_features, user_features=user_features).mean()

print(f'Precision: {precision}')

recall = recall_at_k(model, num_threads=NUM_THREADS, k=10,
                     test_interactions=test,train_interactions=train,
                     item_features=item_features, user_features=user_features).mean()

print(f'Recall: {recall}')

auc = auc_score(model, num_threads=NUM_THREADS,
                      test_interactions=test,train_interactions=train,
                      item_features=item_features, user_features=user_features).mean()
print(f'AUC: {auc}')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section7"></a>
## <font color="#92002A">7 - Predicción</font>

<br>

Por último, en esta última sección hemos creado una función para realizar recomendaciones a los usuarios del conjunto de datos empleado. Tras su ejecución, mostrara qué libros le gustan al usuario/s y cuales podemos recomendarle/s.

<br>

Las predicciones las realizaremos mediante la función ***predict***, indicando los usuarios a los que queremos realizar las recomendaciones:

In [20]:
book_dict = dict([(x['ISBN'], (x['Book-Title'], x['Book-Author']))
                  for x in goodbooks.get_book_features()])

In [21]:
def sample_recommendation(model, data, user_ids, u_features=None, n_threads=1):

    #number of users and books in training data
    n_users, n_items = data.shape

    #generate recommendations for each user we input
    for user_id in user_ids:

        #books they already like
        known_positives = item_labels[data.tocsr()[user_id].indices]

        #books our model predicts they will like
        scores = model.predict(user_id, np.arange(n_items), user_features=u_features, num_threads=n_threads)
        #rank them in order of most liked to least
        top_items = item_labels[np.argsort(-scores)]

        #print out the results
        print("User %s" % user_id)
        print("  Known positives:")

        for x in known_positives[:5]:
            if x in book_dict:
                print(f"    {book_dict[x][0]} ({book_dict[x][1]})")
            else:
                print(f"    {x}")

        print("  Recommended:")

        for x in top_items[:5]:
            if x in book_dict:
                print(f"    {book_dict[x][0]} ({book_dict[x][1]})")
            else:
                print(f"    {x}")
        print()

In [22]:
sample_recommendation(model, interactions, [2342,9567,6756], u_features=user_features, n_threads=NUM_THREADS)

User 2342
  Known positives:
    Wayside School is Falling Down (Louis Sachar)
    Watermelon (Marian Keyes)
    Lucy Sullivan Is Getting Married (Marian Keyes)
    Smart Vs. Pretty (Valerie Frankel)
    The Princess Diaries (Meg Cabot)
  Recommended:
    0140296794
    Wild Animus (Rich Shapero)
    The Keys to the Street (Ruth Rendell)
    0850791154
    Berlin Diaries, 1940-1945 (Marie Vassiltchikov)

User 9567
  Known positives:
    Dancing on Air (Harper Monogram) (Susan Wiggs)
    Comeback (Dick Francis)
    How the Garcia Girls Lost Their Accents (Plume Contemporary Fiction) (Julia Alvarez)
  Recommended:
    Shameless (Jennifer Blake)
    Berlin Diaries, 1940-1945 (Marie Vassiltchikov)
    Acid Row (Minette Walters)
    Hemlock Bay (Catherine Coulter)
    The Long Silence of Mario Salviati : A Novel (Etienne van Heerden)

User 6756
  Known positives:
    Being There (Jerzy Kosinski)
    I Know Why the Caged Bird Sings (MAYA ANGELOU)
  Recommended:
    0099255189
    Terminal (R

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section8"></a>
## <font color="#92002A">8 - Enlaces de interés</font>

<br>

A continuación, hemos adjuntado una serie de enlaces que hemos empleado para realizar esta libreta:

* ["Build a Machine Learning Recommender" (*James Hope*)](https://towardsdatascience.com/build-a-machine-learning-recommender-72be2a8f96ed).
* ["Recommendation Systems - Learn Python for Data Science" (*Siraj Raval*)](https://www.youtube.com/watch?v=9gBC9R-msAk) [Video].
* ["Building datasets" (*LightFM*)](https://making.lyst.com/lightfm/docs/examples/dataset.html).

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>